In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1569793406175_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, isnull
from pyspark.sql.functions import (year, month, dayofmonth,
                                   hour, weekofyear, date_format)
from pyspark.sql.functions import monotonically_increasing_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://rcr-udacity-dend-project3/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# get filepath to song data file
song_data = os.path.join(input_data, "song_data/*/*/*/*.json")

# read song data file
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [6]:
# extract columns to create songs table
songs_table = df.select(['song_id', 'title',
                        'artist_id', 'year', 'duration'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
songs_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [8]:
songs_table.dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table.write \
    .partitionBy("year", "artist_id") \
    .parquet(F"{output_data}/songs/", mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# extract columns to create artists table
df = df \
    .withColumnRenamed('artist_name', 'name') \
    .withColumnRenamed('artist_location', 'location') \
    .withColumnRenamed('artist_latitude', 'latitude') \
    .withColumnRenamed('artist_longitude', 'longitude')

artists_table = df.select(['artist_id',
                           'name',
                           'location',
                           'latitude',
                           'longitude'])

In [ ]:
artists_table.count()

In [ ]:
artist_table = artists_table.dropDuplicates()

In [ ]:
artists_table.count()

In [ ]:
# write artists table to parquet files
artists_table.write.parquet(F"{output_data}/artists/", mode='overwrite')

# Log Data

In [ ]:
# get filepath to log data file
log_data = os.path.join(input_data, "log_data/*/*/*.json")

In [ ]:
# read log data file
df_log_data = spark.read.json(log_data)

In [ ]:
df_log_data.count()

In [ ]:
# filter by actions for song plays
df_log_data = df_log_data \
    .withColumnRenamed('userId', 'user_id') \
    .withColumnRenamed('firstName', 'first_name') \
    .withColumnRenamed('lastName', 'last_name')

# extract columns for users table
users_table = df_log_data.select(['user_id',
                                  'first_name',
                                  'last_name',
                                  'gender',
                                  'level'])

In [ ]:
users_table.count()

In [ ]:
users_table.dropDuplicates()

In [ ]:
users_table.count()

In [ ]:
# write users table to parquet files
users_table.write.parquet(F"{output_data}/users/", mode='overwrite')

In [ ]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000))
df_log_data = df_log_data .withColumn(
            'timestamp', get_timestamp(df_log_data['ts']))

In [ ]:
# create datetime column from original timestamp column
get_datetime = udf(
        lambda x: datetime.fromtimestamp(x/1000)
        .strftime('%Y-%m-%d %H:00:00'))

df_log_data = df_log_data .withColumn(
            'datetime', get_datetime(df_log_data['ts']))

In [ ]:
# extract the day of the week from the timestamp
get_weekday = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%w')))

In [ ]:
# extract columns to create time table
df_log_data = df_log_data \
    .withColumn('hour', hour(df_log_data['datetime'])) \
    .withColumn('day', dayofmonth(df_log_data['datetime'])) \
    .withColumn('week', weekofyear(df_log_data['datetime'])) \
    .withColumn('month', month(df_log_data['datetime'])) \
    .withColumn('year', year(df_log_data['datetime'])) \
    .withColumn('weekday', get_weekday(df_log_data['ts']))

time_table = df_log_data.select(
        ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']
        )
time_table = time_table.dropDuplicates()

# time_table = time_table.filter(time_table.timestamp != Null)

In [ ]:
time_table.count()

In [ ]:
time_table.dropDuplicates().count()

In [ ]:
# write time table to parquet files partitioned by year and month
time_table.write \
          .partitionBy("year", "month") \
          .parquet(F"{output_data}/time/", mode='overwrite')

## Song data

In [ ]:
# read in song data to use for songplays table
song_data = os.path.join(input_data, "song_data/*/*/*/*.json")
df_song = spark.read.json(song_data)

df_song.count()

In [ ]:
df_song_data = df_song.withColumnRenamed('year', 'song_year')

In [ ]:
df_log_data.count()

In [ ]:
df_log_data = df_log_data.filter(df_log_data.page == 'NextSong')

In [ ]:
df_log_data.count()

In [ ]:
# extract columns from joined song and log to create songplays table
df_joined = df_song_data.join(
                df_log_data,
                (df_song_data.artist_name == df_log_data.artist)
                & (df_song_data.title == df_log_data.song),
                "inner")
df_joined.count()

In [ ]:
df_joined = df_joined \
    .withColumn("songplay_id", monotonically_increasing_id()) \
    .withColumnRenamed('sessionId', 'session_id') \
    .withColumnRenamed('userAgent', 'user_agent') \
    .withColumnRenamed('timestamp', 'start_time')

In [ ]:
df_joined.printSchema()

In [ ]:
songplays_table = df_joined.select(['month', 'year', 'songplay_id',
                                    'start_time', 'user_id', 'level',
                                    'song_id', 'artist_id',
                                    'session_id', 'location',
                                    'user_agent'])

In [ ]:
songplays_table.take(2)

In [ ]:
songplays_table.count()

In [ ]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write \
               .partitionBy("year", "month") \
               .parquet(F"{output_data}/songplays/", mode='overwrite')

In [ ]:
schema = StructType([
	StructField("artist", StringType(),True),
	StructField("auth", StringType(),True),
	StructField("firstName", StringType(),True),
	StructField("gender", StringType(),True),
	StructField("itemInSession", LongType(),True),
	StructField("lastName", StringType(),True),
	StructField("length", DoubleType(),True),
	StructField("level", StringType(),True),
	StructField("location", StringType(),True),
	StructField("method", StringType(),True),
	StructField("page", StringType(),True),
	StructField("registration", DoubleType(),True),
	StructField("sessionId", LongType(),True),
	StructField("song", StringType(),True),
	StructField("status", LongType(),True),
	StructField("ts", LongType(),True),
	StructField("userAgent", StringType(),True),
	StructField("userId", StringType(),True)])

In [ ]:
#log_data = os.path.join(input_data, "log_data/")
df = spark.read.json(log_data, schema=schema)
# df_log_data = spark.read.json(log_data)